1.Import the libraries, functions

In [182]:
import pandas as pd
import numpy as np
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


2.Load the List of postal codes of Toronto

In [183]:
data1 = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',skiprows=1)[0]
data1

,M1A,Not assigned,Not assigned.1
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"
...,...,...,...
174,M5Z,Not assigned,Not assigned
175,M6Z,Not assigned,Not assigned
176,M7Z,Not assigned,Not assigned
177,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


3.Create and clean dataframe 

Rename the columns like ['PostalCode', 'Borough', 'Neighborhood']

In [184]:
data1=data1.rename({'M1A': 'PostalCode','Not assigned' : 'Borough', 'Not assigned.1':'Neighborhood'}, axis ='columns')

In [185]:
data1


,PostalCode,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"
...,...,...,...
174,M5Z,Not assigned,Not assigned
175,M6Z,Not assigned,Not assigned
176,M7Z,Not assigned,Not assigned
177,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [186]:
data1.shape

(179, 3)

In [188]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(data1['Borough'].unique()),
        data1.shape[0]
    )
)

The dataframe has 11 boroughs and 179 neighborhoods.


Ignore cells with a Borough that is Not assigned

In [189]:
#filter my df with loc.The "~" means we want the opposit of the condition. 
#The ":" means we want to keep all the columns
data1_new=data1.loc[~(data1['Borough']=='Not assigned'), :].reset_index(drop=True)
data1_new

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Check if Neighborhood are null

In [181]:
data1_new[data1_new['Neighborhood'].isnull()]

,PostalCode,Borough,Neighborhood,rating


In [151]:
data1_new.shape

(103, 3)

Update Not assigned Neighborhood with value of Borough

In [190]:
data1_new['Neighborhood']=np.where(data1_new['Neighborhood']=='Not assigned',data1_new['Borough'],data1_new['Neighborhood'])
        


In [191]:
data1_new

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [192]:
data1_new.shape

(103, 3)